<a href="https://colab.research.google.com/github/edan-ais/Hubbalicious-Products/blob/main/Label_Updater_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# Install + Import Dependencies
# ==============================
!pip install --quiet PyMuPDF google-api-python-client google-auth-httplib2 google-auth-oauthlib

import os
import io
import datetime
import tempfile
import fitz  # PyMuPDF

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

# ==============================
# Authenticate to Google
# ==============================
auth.authenticate_user()
drive_service = build('drive', 'v3')

# ==============================
# Folder IDs
# ==============================
UPDATING_LABELS_FOLDER_ID = "1eH6TKIzygZOLdqvM3rwktV0RgsZR9CID"
ARCHIVE_FOLDER_ID = "1DgzQzMyZi25zMGUb3RHup8OpkILLYnml"

# ==============================
# Google Drive Helpers
# ==============================
def list_files_in_folder(folder_id):
    """Return all files in a folder (handles pagination)."""
    files = []
    page_token = None
    while True:
        query = f"'{folder_id}' in parents and trashed=false"
        results = drive_service.files().list(
            q=query,
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token
        ).execute()
        files.extend(results.get("files", []))
        page_token = results.get("nextPageToken")
        if not page_token:
            break
    return files

def download_file_to_path(file_id, local_path):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.close()

def upload_file_replace(file_id, local_path, mimetype="application/pdf"):
    media = MediaFileUpload(local_path, mimetype=mimetype, resumable=True)
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media
    ).execute()
    return updated_file

def find_file_in_folder_by_name(folder_id, name):
    safe_name = name.replace('"', '\\"')
    query = f"'{folder_id}' in parents and name = \"{safe_name}\" and trashed=false"
    res = drive_service.files().list(q=query, fields='files(id, name)', spaces='drive').execute()
    files = res.get('files', [])
    return files[0] if files else None

def copy_file_to_folder(file_id, new_folder_id, new_name=None):
    file = drive_service.files().get(fileId=file_id, fields='name').execute()
    name = new_name if new_name else file['name']

    existing = find_file_in_folder_by_name(new_folder_id, name)
    if existing:
        drive_service.files().delete(fileId=existing['id']).execute()

    copied_file = {'name': name, 'parents': [new_folder_id]}
    return drive_service.files().copy(fileId=file_id, body=copied_file).execute()

# ==============================
# Date Utilities
# ==============================
def compute_best_by_date():
    target = datetime.date.today() + datetime.timedelta(days=75)
    first = target.replace(day=1)
    fifteenth = target.replace(day=15)
    dist_first = abs((target - first).days)
    dist_fifteenth = abs((target - fifteenth).days)
    if dist_first < dist_fifteenth:
        rounded = first
    elif dist_fifteenth < dist_first:
        rounded = fifteenth
    else:
        rounded = first if first < fifteenth else fifteenth
    return rounded.strftime("%m/%d/%Y")

# ==============================
# PDF Text Replacement
# ==============================
def replace_best_by_text(doc, new_date):
    replaced = False
    for page_num, page in enumerate(doc, start=1):
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            for l in b.get("lines", []):
                for s in l.get("spans", []):
                    text = s.get("text", "")
                    if text.strip().startswith("Best if used by:"):
                        old_text = text
                        new_text = f"Best if used by: {new_date}"
                        bbox = fitz.Rect(s["bbox"])
                        # Determine rotation: vertical if taller than wide
                        rotation = 90 if bbox.height > bbox.width else 0
                        # White-out old text
                        page.draw_rect(bbox, color=(1,1,1), fill=(1,1,1))

                        # Bottom-left reference for both horizontal and vertical
                        if rotation == 0:
                            x, y = bbox.x0, bbox.y1
                        else:
                            # Vertical: rotate around bottom-right of original bbox
                            x, y = bbox.x1, bbox.y1

                        page.insert_text(
                            (x, y),
                            new_text,
                            fontname="helv",
                            fontsize=s["size"],
                            color=(0,0,0),
                            rotate=rotation
                        )
                        print(f"Replaced on page {page_num}: '{old_text}' → '{new_text}' (rotation={rotation})")
                        replaced = True
    return replaced

# ==============================
# Main Processing
# ==============================
def process_labels():
    files = list_files_in_folder(UPDATING_LABELS_FOLDER_ID)
    # robust PDF filtering
    pdf_files = [f for f in files if f.get('mimeType') == 'application/pdf' or f['name'].strip().lower().endswith('.pdf')]

    target_date = compute_best_by_date()
    print(f"Target best-by date: {target_date}\n")

    summary = []

    for f in pdf_files:
        file_id = f["id"]
        name = f["name"]
        print(f"Processing: {name} (id: {file_id})")

        with tempfile.TemporaryDirectory() as tmpdir:
            local_path = os.path.join(tmpdir, name)
            # Download original
            download_file_to_path(file_id, local_path)
            print(" - downloaded original")
            # Archive original
            copy_file_to_folder(file_id, ARCHIVE_FOLDER_ID, name)
            print(f" - archived original as {name}")
            # Open and replace text
            doc = fitz.open(local_path)
            replaced = replace_best_by_text(doc, target_date)

            if replaced:
                # Save to temp file then overwrite original
                new_path = local_path + "_updated.pdf"
                doc.save(new_path, deflate=True)
                doc.close()
                os.replace(new_path, local_path)
                # Upload back to Drive
                upload_file_replace(file_id, local_path)
                print(" - updated original with new text\n")
                summary.append((name, "updated", target_date))
            else:
                doc.close()
                print(" - no matches found to replace\n")
                summary.append((name, "no-replace", None))

    print("\n=== Run summary ===")
    for item in summary:
        print(item)

# ==============================
# Run once
# ==============================
process_labels()

Target best-by date: 11/15/2025

Processing: 2.5 x 4 Birthday Cake Chardonnay Fudge Label.pdf (id: 1rdkeV_1QycaOo2DXM4RBfcGrbNCfN8O6)
 - downloaded original
 - archived original as 2.5 x 4 Birthday Cake Chardonnay Fudge Label.pdf
Replaced on page 1: 'Best if used by: 10/08/2025' → 'Best if used by: 11/15/2025' (rotation=90)
Replaced on page 1: 'Best if used by: 11/15/2025' → 'Best if used by: 11/15/2025' (rotation=90)
 - updated original with new text

Processing: 2.5 x 4 Birthday Cake Fudge Label.pdf (id: 1M_jgaCig5rSZXfpkFCI0tE3GOVe_U6vA)
 - downloaded original
 - archived original as 2.5 x 4 Birthday Cake Fudge Label.pdf
Replaced on page 1: 'Best if used by: 10/08/2025' → 'Best if used by: 11/15/2025' (rotation=90)
Replaced on page 1: 'Best if used by: 11/15/2025' → 'Best if used by: 11/15/2025' (rotation=90)
 - updated original with new text

Processing: 2.5 x 4 Butterfinger Fudge Label.pdf (id: 1RPj7btH7qRiZoHloC3dMA0KVm2mGzFBc)
 - downloaded original
 - archived original as 2.5 x